In [ ]:
from preprocess import drop_useless

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
data_test=pd.read_csv('../../data/test_airbnb_berlin.xls')
data_train=pd.read_csv('../../data/train_airbnb_berlin.xls')

In [ ]:
#Data visualisation

# computing number of rows
rows = len(data_train.axes[0])
print('number row :',rows)
# computing number of columns
cols = len(data_train.axes[1])
print('number columns :',cols)

In [ ]:
data_train.columns.values

In [ ]:
data_train.head(1)

In [ ]:
data_train=drop_useless(data_train)

__Analyse du taux de missing par feature__

In [ ]:
#liste des features avec un taux de "missing" moyen (~18%)
listfeatures=[] 

def missing(df):
    total = 0
    for col in df.columns:
        miss = df[col].isnull().sum()
        pct = df[col].isna().mean() * 100
        total += miss
        if miss != 0:
            print('{} => {} [{}%]'.format(col, df[col].isnull().sum(), round(pct, 2)))
            if pct>15 and pct <25 :
                listfeatures.append(col)
        #else:
        #    print('{} => {} '.format(col, 'no missing value'))

    if total == 0:
        print("no missing values")
        
missing(data_train)

__Gestion des features à taux de "missing" extrême__

Lorsque l'on regarde les features où il manque moins de 30 données, on se rend compte que l'absence de ces données est MCAR. On se permet donc de supprimer les lignes où ces données sont manquantes. 
Il y a une exception sur la feature 'Is Superhost': si pour ces host il y a une note générale au dessus de 80, on les catégorises en Superhost, sinon, non. 

In [ ]:
#Suppression des features à taux de missing trop élevé 
data_train=data_train.drop(['Square Feet','Host Response Time','Host Response Rate'], axis=1)

In [ ]:
#Listwise deletion pour les features à taux de missing faible (borne max des rows concernés : 60 /15k = 0.4%)
for j in ['Price','Bathrooms','Bedrooms','Beds']:
    for i in data_train[data_train[j].isna()== True].index:
        data_train=data_train.drop(i, axis=0)

In [ ]:
# Gestion des 'Is Superhost' : si le host est noté, et ce au dessus de 80, il est supposé superhost, sinon il ne l'est pas
for i in data_train[data_train['Is Superhost'].isna()== True].index:
    if data_train.loc[i,'Overall Rating']>=80:
        data_train.loc[i,'Is Superhost']='t'
    else:
        data_train.loc[i,'Is Superhost']='f'

__Recherche de corrélations entre les features présentant un taux de missing value "NaN" autour de 18%__

On va s'aider de la liste **listfeatures** définie plus tôt


In [ ]:
#tableau de correlation de "missing" entre deux features
tablefeatures=np.ones((len(listfeatures),len(listfeatures)))

#calcul des corrélations de missing entre deux features
index1=0 
for q in listfeatures:
    nanrows=len(data_train[data_train[q].isna()== True].axes[0])
    index2=0
    for p in listfeatures :
        if q!=p:
            bothmiss=len(data_train[data_train[q].isna()== True][data_train[p].isna()== True].axes[0])
        #print ("P("+q+" et "+p+" vaut NaN) =" + str(bothmiss/nanrows))
            tablefeatures[index1,index2]=bothmiss/nanrows
        index2+=1
    index1+=1   

#affichage du tableau de corrélations
print("Pourcentage minimal de corrélation entre deux features de la liste : " + str(round(tablefeatures.min()*100,2)) + "% \n")
print(tablefeatures)

On peut donc raisonnablement affirmer que ces features sont de type MNAR : une listwise deletion n'est donc pas conseillée (et avec 18% d'instances concernées, ce n'est pas idéal)

__Enregistrement__

In [ ]:
from pandas import NA
data_train.replace("*", NA)

In [ ]:
#fichier csv où l'on sans les ratings 
data_train_clean=data_train.drop(['Overall Rating', 'Accuracy Rating', 'Cleanliness Rating',
       'Checkin Rating', 'Communication Rating', 'Location Rating','Value Rating'], axis=1)
data_train_clean.to_csv('../../data/data_train_clean.csv',index=False)

#fichier csv où l'on garde les ratings 

for j in ['Overall Rating', 'Accuracy Rating', 'Cleanliness Rating','Checkin Rating', 'Communication Rating', 'Location Rating','Value Rating']:
       for i in data_train_clean_ratings_included[data_train_clean_ratings_included[j].isna()== True].index:
              data_train_clean_ratings_included=data_train_clean_ratings_included.drop(i, axis=0)

data_train_clean_ratings_included.to_csv('../../data/data_train_clean_ratings_included.csv',index=False)
